# Deploy a Serverless XGBoost Model Server
  --------------------------------------------------------------------

The following notebook demonstrates how to deploy an XGBoost model server (a.k.a <b>Nuclio-serving</b>)

#### **notebook how-to's**
* Write and test model serving class in a notebook.
* Deploy the model server function.
* Invoke and test the serving function.

<a id="top"></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[test the model serving class locally](#test-locally)**<br>
**[deploy our serving class using as a serverless function](#deploy)**<br>
**[test our model server using HTTP request](#test-model-server)**<br>

In [7]:
# nuclio: ignore
import nuclio 

<a id="define-function"></a>
### **define a new function and its dependencies**

In [8]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=XGBoostModel

%nuclio config spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


## Function Code

In [9]:
# import kfserving
import os
import json
import numpy as np
import xgboost as xgb
from cloudpickle import load

### Model Serving Class

import mlrun
class XGBoostModel(mlrun.runtimes.MLModelServer):
    def load(self):
        model_file, model_obj = self.get_model()
        self.model = load(open(str(model_file), "rb"))

    def predict(self, body):
        try:
            feats = np.asarray(body["instances"], dtype=np.float32).reshape(-1, 4)
            result = self.model.predict(feats, validate_features=False)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [10]:
# nuclio: end-code

<a id="test-locally"></a>
## Test the function locally

The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

> **Verify there is a model file in the model_dir path (generated by the training notebook)**

In [11]:
from mlrun import mlconf
model_dir = os.path.join(mlconf.artifact_path, "models")

my_server = XGBoostModel("my-model", model_dir=model_dir)
my_server.load()

In [12]:
REPO_URL = "https://raw.githubusercontent.com/yjb-ds/testdata/master"
DATA_PATH = "data/classifier-data.csv"
MODEL_PATH = "models/xgb_test"

In [13]:
import pandas as pd
xtest = pd.read_csv(f"{REPO_URL}/{DATA_PATH}")

We can use the `.predict(body)` method to test the model.

In [14]:
import json, numpy as np
preds = my_server.predict({"instances":xtest.values[:10,:-1].tolist()})

In [15]:
print("predicted class:", preds)

predicted class: [0, 0, 0, 1, 1, 1, 0, 0, 1, 0]


<a id="deploy"></a>
### **deploy our serving class using as a serverless function**
in the following section we create a new model serving function which wraps our class , and specify model and other resources.

the `models` dict store model names and the assosiated model **dir** URL (the URL can start with `S3://` and other blob store options), the faster way is to use a shared file volume, we use `.apply(mount_v3io())` to attach a v3io (iguazio data fabric) volume to our function. By default v3io will mount the current user home into the `/User` function path.

**verify the model dir does contain a valid model file**

In [16]:
from mlrun import new_model_server
from mlrun.platforms.other import auto_mount
import requests

In [17]:
fn = new_model_server("xgb-test",
                      model_class="XGBoostModel",
                      models={"xgb_serving_v2": f"{model_dir}"})
fn.spec.description = "xgboost test data classification server"
fn.metadata.categories = ["serving", "ml"]
fn.metadata.labels = {"author": "yaronh", "framework": "xgboost"}

fn.export("function.yaml")
fn.apply(auto_mount())

[mlrun] 2020-06-09 13:49:07,929 function spec saved to path: function.yaml


## tests

In [18]:
addr = fn.deploy(project="xgboost")

[mlrun] 2020-06-09 13:49:08,955 deploy started
[nuclio] 2020-06-09 13:49:10,036 (info) Build complete
[nuclio] 2020-06-09 13:49:16,353 (info) Function deploy complete
[nuclio] 2020-06-09 13:49:16,477 done updating xgboost-xgb-test, function address: 3.23.82.202:30865


In [19]:
addr

'http://3.23.82.202:30865'

<a id="test-model-server"></a>
### **test our model server using HTTP request**


We invoke our model serving function using test data, the data vector is specified in the `instances` attribute.

In [20]:
# KFServing protocol event
event_data = {"instances": xtest.values[:10,:-1].tolist()}

In [21]:
import json
resp = requests.put(addr + "/xgb_serving_v2/predict", json=json.dumps(event_data))

# mlutils function for this?
tl = resp.text.replace("[","").replace("]","").split(",")
#assert preds == [int(i) for i in np.asarray(tl)]

In [22]:
tl

['0', ' 0', ' 0', ' 1', ' 1', ' 1', ' 0', ' 0', ' 1', ' 0']

In [23]:
preds

[0, 0, 0, 1, 1, 1, 0, 0, 1, 0]

**[back to top](#top)**